# Utilizando o OCI Dataflow Studio

### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

#### Conectando ao Dataflow Studio

In [ ]:
# importando a biblioteca ADS e realizando a autenticação
import ads

ads.set_auth("resource_principal")

In [ ]:
%load_ext dataflow.magics

In [ ]:
%use_session -s '' -f

In [ ]:
%%spark
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from delta import *
from pyspark.sql.functions import *
from datetime import datetime

#### Interagindo com arquivos utilizando Dataflow Studio

In [ ]:
%%spark
df_nyc_tlc = spark.read.parquet("oci://tlc@grkqi1ha7xky/*.parquet", header=False, inferSchema=True)
df_nyc_tlc.show(3)
df_nyc_tlc.createOrReplaceTempView("nyc_tlc")

#### Convertendo para um pandas dataframe

In [ ]:
%%spark -c sql -o df_nyc_tlc
SELECT vendorID, passenger_count, trip_distance, payment_type FROM nyc_tlc LIMIT 1000;

In [ ]:
df_nyc_tlc.head()

#### Interagindo com hive metastore

In [ ]:
%%spark
spark.sql("CREATE DATABASE studio")

In [ ]:
%%spark 
spark.sql("SHOW DATABASES").show()

In [ ]:
%%spark
df_nyc_tlc.write.format("delta").mode("overwrite").saveAsTable("studio.tlc_table")

#### Trabalhando com Delta Lake

In [ ]:
%%spark
df_nyc_tlc.write.format("delta").mode("overwrite").save("oci://raw-data@<seu_namespace>/tlc_files/")

In [ ]:
%%spark
read_delta = DeltaTable.forPath(spark, 'oci://raw-data@<seu_namespace>/tlc_files/')

In [ ]:
%%spark
read_delta.delete("VendorID < 2")
read_delta.toDF().groupBy("VendorID").sum("total_amount").show()

In [ ]:
%%spark
df = spark.read.format("delta") \
  .option("versionAsOf", 1) \
  .load("oci://raw-data@<seu_namespace>/tlc_files/")

df.groupBy("VendorID").sum("total_amount").show()

In [ ]:
%%spark
df = spark.read.format("delta") \
  .option("versionAsOf", 0) \
  .load("oci://raw-data@<seu_namespace>/tlc_files/")

df.groupBy("VendorID").sum("total_amount").show()

In [ ]:
%%spark
read_delta.restoreToVersion(0)

spark.read.format("delta").load("oci://raw-data@<seu_namespace>/tlc_files/").groupBy("VendorID").sum("total_amount").show()

In [ ]:
%%spark
read_delta.history().show()